# Services (hour)

In [1]:
import pandas as pd
import dask.dataframe as dd

ModuleNotFoundError: No module named 'dask'

## Database connection

In [2]:
import yaml
from sqlalchemy import create_engine


with open("../config.yml", "r") as file:
	config = yaml.safe_load(file)
	config_OLTP = config["OLTP"]
	config_OLAP = config["OLAP"]


url_OLTP = (f"{config_OLTP['drivername']}://{config_OLTP['user']}:{config_OLTP['password']}"
			f"@{config_OLTP['host']}:{config_OLTP['port']}/{config_OLTP['database_name']}")

url_OLAP = (f"{config_OLAP['drivername']}://{config_OLAP['user']}:{config_OLAP['password']}"
			f"@{config_OLAP['host']}:{config_OLAP['port']}/{config_OLAP['database_name']}")

OLTP_connection = create_engine(url_OLTP)
OLAP_connection = create_engine(url_OLAP)

## Extraction

In [3]:
# Extraction
service_fact = pd.read_sql_table("mensajeria_servicio", OLTP_connection)
service_status_fact = pd.read_sql_table("mensajeria_estadosservicio", OLTP_connection)
time_dimension = pd.read_sql_table("TIME_DIMENSION", OLAP_connection)
courier_dimension = pd.read_sql_table("COURIER_DIMENSION", OLAP_connection)
customer_dimension = pd.read_sql_table("CUSTOMER_DIMENSION", OLAP_connection)
office_dimension = pd.read_sql_table("OFFICE_DIMENSION", OLAP_connection)
service_status_dimension = pd.read_sql_table(
    "SERVICE_STATUS_DIMENSION", OLAP_connection
)

## Transformation

In [4]:
# Transformation
# 1. Combinar 'fecha_solicitud' y 'hora_solicitud' para crear 'request_time'
service_fact["request_time"] = pd.to_datetime(
    service_fact["fecha_solicitud"].astype(str)
    + " "
    + service_fact["hora_solicitud"].astype(str)
)

In [5]:
# 2. Extraer la hora de 'request_time'
service_fact["request_hour"] = service_fact["request_time"].dt.hour

In [6]:
# 3. Merge con time_dimension para obtener time_id
service_hour_fact_table = pd.merge(
    service_fact,
    time_dimension[["time_id", "hour"]],
    left_on="request_hour",
    right_on="hour",
    how="left",
)

In [7]:
# 4. Merge con las dimensiones necesarias para obtener los IDs correspondientes
service_hour_fact_table = pd.merge(
    service_hour_fact_table,
    customer_dimension[["customer_id"]],
    left_on="cliente_id",
    right_on="customer_id",
    how="left",
)

In [8]:
service_hour_fact_table = pd.merge(
    service_hour_fact_table,
    courier_dimension[["courier_id"]],
    left_on="mensajero_id",
    right_on="courier_id",
    how="left",
)

In [ ]:
service_hour_fact_table = pd.merge(
    service_hour_fact_table,
    office_dimension[["office_id"]],
    left_on="origen_id",
    right_on="office_id",
    how="left",
)

: 

In [ ]:
# 5. Merge con service_status_fact para obtener estado_id
service_hour_fact_table = pd.merge(
    service_hour_fact_table,
    service_status_fact[["servicio_id", "estado_id"]],
    left_on="id",
    right_on="servicio_id",
    how="left",
)

## Load

In [ ]:
## Cargar la Tabla de Hechos Horaria
# service_hour_fact_table.to_sql(
# 		"SERVICE_HOUR_FACT_TABLE", OLAP_connection, if_exists="replace", index=False
# )